<a href="https://colab.research.google.com/github/suryagokul/Data-Science-Portfolio/blob/master/Feature%20Engineerin%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Categorical Variables

1.   Mode Imputation
2.   Capture NaN values with new feature
3.   One Hot Encoding
4.   Ordinal Encoding
5.   Target Guided Ordinal Encoding
6.   Mean Encoding
7.   Probability Ratio Encoding








In [423]:
import seaborn as sns
import pandas as pd
import numpy as np

from seaborn import load_dataset

In [424]:
df = load_dataset('titanic')

df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Columns: 15 entries, survived to alone
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [425]:
df[:5]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [426]:
df.describe(include='all').head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
count,891.000000,891.000000,891,714.000000,891.000000,891.000000,891.000000,889,891,891,891,203,889,891,891
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,7,3,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,C,Southampton,no,True
freq,NaN,NaN,577,NaN,NaN,NaN,NaN,644,491,537,537,59,644,549,537
mean,0.383838,2.308642,NaN,29.699118,0.523008,0.381594,32.204208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [427]:


Categoricals = df.select_dtypes(exclude=['int64','float64'])

Categoricals.isnull().sum()

sex              0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

# MODE IMPUTATION

In [428]:
def impute_nan(df,feature):
  df["new"+feature] = np.where(df[feature].isnull(),df[feature].mode(),df[feature])

for feature in ['embarked','embark_town']:
  impute_nan(df,feature)

In [429]:
df[['newembarked','newembark_town']].isnull().sum()

newembarked       0
newembark_town    0
dtype: int64

# Capture NaN values with new feature




*   Using one's and zeroe's
*   Creating New Category





In [430]:
df1 = load_dataset('titanic')

In [431]:
Categoricals.isnull().sum()

sex              0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

**Using one's and zeroe's**

In [432]:
df['deck_new'] = np.where(df1['deck'].isnull(),1,0)

In [433]:
print(df['deck_new'].isnull().sum())

df[:3]

0


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,newembarked,newembark_town,deck_new
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,S,Southampton,1
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,C,Cherbourg,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,S,Southampton,1


**Creating New Category**

In [434]:
df['deck_newCat'] = np.where(df1['deck'].isnull(),"Missing_Category",df['deck'])


In [435]:
df[:3]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,newembarked,newembark_town,deck_new,deck_newCat
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,S,Southampton,1,Missing_Category
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,C,Cherbourg,0,C
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,S,Southampton,1,Missing_Category


# One Hot Encoding

In [436]:
df2 = load_dataset('titanic')

In [437]:
Categoricals.isnull().sum()

sex              0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [438]:
df2['deck'].unique()

[NaN, 'C', 'E', 'G', 'D', 'A', 'B', 'F']
Categories (7, object): ['C', 'E', 'G', 'D', 'A', 'B', 'F']

In [439]:
def impute_nan(df2,feature):
   df2 = pd.concat([df2,pd.get_dummies(df2[['embarked','embark_town','deck']])],axis=1)


for feature in ['embarked','embark_town','deck'] :
  impute_nan(df2,feature)


In [440]:

df2[:3]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


**Problem**

> But sometimes One Hot Encoding leads to Curse Of Dimesnionality that means As we increase our features like 200,300... then our model performance will decreases.





**Solution**



   We have to take top 10 or 20 most frequent one and we reject remaining all.
 
  It is proved in One of the kaggle competiton said to be KDD Orange Cup.





In [441]:
df2['deck'].isnull().sum()

688

Here there are only 7 categories in deck.So, we taking top 3 repeated categories.
But in real life we may have a scenario like 100-150 categories at that it is more useful..

In [442]:
len(df2['deck'].value_counts())

7

In [443]:
Gp = df2.groupby('deck')['deck'].count().sort_values(ascending=False).head(3).index

top_3_list = list(Gp)

top_3_list


['C', 'B', 'D']

In [444]:
for i in top_3_list:
  df2[i] = np.where(df2['deck']==i,1,0)


In [445]:
df2[:10]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,C,B,D
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,0,0,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1,0,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,0,0,0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1,0,0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,0,0,0
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,0,0,0
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True,0,0,0
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False,0,0,0
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False,0,0,0
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False,0,0,0


# Ordinal Encoding

In [446]:
import datetime


In [447]:
today_date = datetime.datetime.today()

today_date

datetime.datetime(2020, 10, 21, 11, 33, 44, 143536)

In [448]:
days = []
for i in range(20):
  days.append(today_date - datetime.timedelta(i))

In [449]:
days

[datetime.datetime(2020, 10, 21, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 20, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 19, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 18, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 17, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 16, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 15, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 14, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 13, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 12, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 11, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 10, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 9, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 8, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 7, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 6, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 5, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 4, 11, 33, 44, 143536),
 datetime.datetime(2020, 10, 3, 11

In [450]:
days_df = pd.DataFrame(days,columns = ['Day'])

In [451]:
days_df

,Day
0,2020-10-21 11:33:44.143536
1,2020-10-20 11:33:44.143536
2,2020-10-19 11:33:44.143536
3,2020-10-18 11:33:44.143536
4,2020-10-17 11:33:44.143536
5,2020-10-16 11:33:44.143536
6,2020-10-15 11:33:44.143536
7,2020-10-14 11:33:44.143536
8,2020-10-13 11:33:44.143536
9,2020-10-12 11:33:44.143536


In [452]:
type(days_df['Day'])

pandas.core.series.Series

In [453]:
days_df['Day'].dt.day

0     21
1     20
2     19
3     18
4     17
5     16
6     15
7     14
8     13
9     12
10    11
11    10
12     9
13     8
14     7
15     6
16     5
17     4
18     3
19     2
Name: Day, dtype: int64

In [454]:
days_df['days'] = days_df['Day'].dt.day_name()

In [455]:
days_df[:5]

,Day,days
0,2020-10-21 11:33:44.143536,Wednesday
1,2020-10-20 11:33:44.143536,Tuesday
2,2020-10-19 11:33:44.143536,Monday
3,2020-10-18 11:33:44.143536,Sunday
4,2020-10-17 11:33:44.143536,Saturday


In [456]:
dic = {'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}

days_df['days_number'] = days_df['days'].map(dic)

In [457]:
days_df[:5]

,Day,days,days_number
0,2020-10-21 11:33:44.143536,Wednesday,2
1,2020-10-20 11:33:44.143536,Tuesday,1
2,2020-10-19 11:33:44.143536,Monday,0
3,2020-10-18 11:33:44.143536,Sunday,6
4,2020-10-17 11:33:44.143536,Saturday,5


# Target Guided Ordinal Encoding

In [526]:
from seaborn import load_dataset

In [532]:
tit_df = load_dataset('titanic')

tit_df[:5]

tit_df['deck'] = np.where(tit_df['deck'].isnull(),"M",tit_df['deck'])

In [533]:
tit_index = tit_df.groupby('deck')['survived'].mean().sort_values().index

tit_index


Index(['M', 'A', 'G', 'C', 'F', 'B', 'E', 'D'], dtype='object', name='deck')

In [534]:
a = [(k,i) for i,k in enumerate(tit_index)]
  

a = dict(a)

a

{'A': 1, 'B': 5, 'C': 3, 'D': 7, 'E': 6, 'F': 4, 'G': 2, 'M': 0}

In [535]:
tit_df['target_ordinal'] = tit_df['deck'].map(a)


In [536]:
tit_df['target_ordinal'] = np.where(tit_df['target_ordinal'].isnull(),"Missing",tit_df['target_ordinal'])

In [537]:
tit_df[:5]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,target_ordinal
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,M,Southampton,no,False,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,3
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,M,Southampton,yes,True,0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,3
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,M,Southampton,no,True,0


# Mean Encoding

In [538]:
tit_index = tit_df.groupby('deck')['survived'].mean().sort_values()

tit_index



deck
M    0.299419
A    0.466667
G    0.500000
C    0.593220
F    0.615385
B    0.744681
E    0.750000
D    0.757576
Name: survived, dtype: float64

In [539]:
tit_df['mean'] = tit_df['deck'].map(tit_index)



tit_df['mean'] = np.where(tit_df['mean'].isnull(),0, tit_df['mean'])


tit_df['mean'].astype('int')


tit_df['mean'] = tit_df['mean'].values.round(2)

In [540]:

tit_df[:5]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,target_ordinal,mean
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,M,Southampton,no,False,0,0.30
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,3,0.59
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,M,Southampton,yes,True,0,0.30
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,3,0.59
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,M,Southampton,no,True,0,0.30


# Probability Ratio Encoding

### Steps (Titanic dataset)


*   Probability of survived based on deck i.e categorical
*   Probability of died based on survived  i.e p(A) = 1-p(B) formuala
*   Finding ratio i.e P(survived)/P(died)
*   Convert ratio to dict and map with deck feature




In [544]:
d_f = load_dataset('titanic')

In [563]:
d_f['deck'] = pd.DataFrame(d_f['deck'].astype('object').fillna("M"))
d_f[:5]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,M,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,M,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,M,Southampton,no,True


In [565]:
prob_df = d_f.groupby('deck')['survived'].mean()

prob_df

deck
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
M    0.299419
Name: survived, dtype: float64

In [567]:
prob_df = pd.DataFrame(prob_df)

In [568]:
prob_df

,survived
deck,
A,0.466667
B,0.744681
C,0.593220
D,0.757576
E,0.750000
F,0.615385
G,0.500000
M,0.299419


In [571]:
prob_df['died'] = 1 - prob_df['survived']

prob_df

,survived,died
deck,,
A,0.466667,0.533333
B,0.744681,0.255319
C,0.593220,0.406780
D,0.757576,0.242424
E,0.750000,0.250000
F,0.615385,0.384615
G,0.500000,0.500000
M,0.299419,0.700581


In [574]:
prob_df['prob_ratio'] = prob_df['survived']/prob_df['died']

prob_df

,survived,died,prob_ratio
deck,,,
A,0.466667,0.533333,0.875000
B,0.744681,0.255319,2.916667
C,0.593220,0.406780,1.458333
D,0.757576,0.242424,3.125000
E,0.750000,0.250000,3.000000
F,0.615385,0.384615,1.600000
G,0.500000,0.500000,1.000000
M,0.299419,0.700581,0.427386


In [576]:
dic = prob_df['prob_ratio'].to_dict()

dic

{'A': 0.875,
 'B': 2.916666666666666,
 'C': 1.4583333333333333,
 'D': 3.125,
 'E': 3.0,
 'F': 1.6000000000000003,
 'G': 1.0,
 'M': 0.4273858921161825}

In [578]:
d_f['deck_encoded'] = d_f['deck'].map(dic)

d_f[:5]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,probabilty_ratio,deck_encoded
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,M,Southampton,no,False,0.427386,0.427386
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1.458333,1.458333
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,M,Southampton,yes,True,0.427386,0.427386
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1.458333,1.458333
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,M,Southampton,no,True,0.427386,0.427386
